In [1]:
from scipy.integrate import quad
from numpy import *
from matplotlib.pylab import *
import matplotlib as mpl
%matplotlib inline
#from pandas.core import *
import pandas as pd

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from sklearn.decomposition import PCA,FastICA

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import pickle


In [2]:
folder = '/mnt/data/spinemodel/conditions/'

In [3]:
#Jupyters: Proper Spine Model - Inhibition.ipynb
with open(folder+"dis_baselinev3_1000_70_65sh.pickle","rb") as f:
    vavgDiS,mesDiS,vtracsDiS,vtracsDDiS,vtracsSDiS,CtracsDiS = pickle.load(f)

with open(folder+"All_baseline_datasetv3_1000_70_65sh.pickle","rb") as f:
    vavgT,mesT,vtracsT,vtracsDT,vtracsST,CtracsT,me2T,_,dataT = pickle.load(f)


In [4]:
labels = ['A1','Dss','Rneck']#,'meanDhead']
labelsmod = labels
labels = ['A1','maxDhead','Dss','Lneck','Lhead','meanDneck_trunc70pc']
labelmorph = labels

In [5]:
pddataT = pd.DataFrame(dataT, columns= list(dataT.keys()))

In [6]:
with open(folder+"dis_ga_glu02v3_1000_70_65sh.pickle","rb") as f:
    dis,mes1,mes01 = pickle.load(f)

with open(folder+"dis_glu_ga02v3_1000_70_65sh.pickle","rb") as f:
    dis,mes,mes0,times,times0 = pickle.load(f)



In [33]:
dis.keys()

Index(['#Sp', '#Bch', 'Dsb', 'Dss', 'd_interSp', 'Vtot', 'Vh', 'Ln', 'SA',
       'nPSD', 'A1', 'A2', 'GPHN_SNR', 'DiS-EMchk', 'Lneck', 'maxDneck',
       'minDneck', 'meanDneck', 'Rneck', 'Lhead', 'maxDhead', 'minDhead',
       'meanDhead', 'Ah', 'Rhead', 'meanDneck_trunc70pc', 'L', 'D', 'AhA0',
       'Rmorph'],
      dtype='object')

In [35]:
pddis = pd.DataFrame(data=dis,columns=dis.keys())


In [36]:
labels = labelsmod+['A2']
datt = dis[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dis[l]))

pddismodel = pd.DataFrame(data=datt,columns=labels)

labels = labelmorph+['A2']
datt = dis[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dis[l]))
datt = dis[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dis[l]))

pddismodel = pd.DataFrame(data=datt,columns=labels)

In [8]:
regressor = LinearRegression()  
#regressor.fit(datt, mesT[:,3])
#coeforig = regressor.coef_*1.0
#print(regressor.coef_)

In [9]:
#y_pred = regressor.predict(datt)

In [10]:
#fullvarlinear = ((mesT[:,3]-y_pred)).var()
#print(fullvarlinear)

In [52]:
def LinReg_varexplain_B(X,y,labels= None, relative = False):
    if labels is None:
        try: 
            labels = X.keys()
        except:
            labels = arange(X.shape[1])
            
    idcs = arange(X.shape[1]).tolist()
    vardat = zeros((X.shape[1],2))
    
    var0 = var(y)
    #y_predt = y*0.0    
    for k in range(X.shape[1]-1):

        varts = zeros(len(idcs))
        #print(idcs)
        for iv,i in enumerate(idcs):
            
            idct = 1*array(idcs)
            idct = delete(idct,iv)
            Xb = X.values[:,array(idct)]
            
            regressor.fit(Xb, y)
            y_pred = regressor.predict(Xb)
            vart = ((y-y_pred)).var()
            
            varts[iv] = vart
        
        #print(varts)
        # We select column that affected the minimum and remove it
        sel = argmin(varts)
        i = idcs[sel]
        vardat[k,:] = (idcs[sel],varts[sel])
        idcs.pop(sel)

    #i = idcs[0]
    vardat[-1,:] = (idcs[0],var(y))
    #print(vardat,var0)
    #hist(y-y_pred,41)

    result = {}
    #print(vardat)
    for i in range(vardat.shape[0]-1,0,-1):
        vsa = (vardat[i,1])/var0
        vs = (vardat[i-1,1])/var0
        #print(labels[int(vardat[i,0])],(vsa-vs)*100)
        result[labels[int(vardat[i,0])]] = vsa-vs
        
    regressor.fit(X, y)
    y_pred = regressor.predict(X)
    vart = ((y-y_pred)).var()
    vsa = (vardat[0,1])/var0
    vs = (vart)/var0
    
    #print(labels[int(vardat[0,0])],(vsa-vs)*100)
    result[labels[int(vardat[0,0])]] = vsa-vs
    result['variance'] = var0
    return(result)

In [53]:
def LinReg_varexplain_F(X,y,labels= None, relative = False):
    if labels is None:
        try: 
            labels = X.keys()
        except:
            labels = arange(X.shape[1])
            
    idcs = arange(X.shape[1]).tolist()
    vardat = zeros((X.shape[1],2))
    
    y_predt = y*0.0
    var0 = var(y)
    minvar = var0
    for k in range(X.shape[1]-1):
        varts = zeros(len(idcs))
        for iv,i in enumerate(idcs):
            regressor.fit(X.values[:,i:(i+1)], y-y_predt)
            y_pred = regressor.predict(X.values[:,i:(i+1)])
            vart = ((y-y_predt-y_pred)).var()
            varts[iv] = vart
        sel = argmin(varts)
        i = idcs[sel]
        regressor.fit(X.values[:,i:(i+1)], y-y_predt)
        y_pred = regressor.predict(X.values[:,i:(i+1)])
        y_predt += y_pred

        vardat[k,:] = (idcs[sel],varts[sel])
        idcs.pop(sel)

    i = idcs[0]
    regressor.fit(X.values[:,i:(i+1)], y-y_predt)
    y_pred = regressor.predict(X.values[:,i:(i+1)])
    vart = ((y-y_predt-y_pred)).var()
    vardat[-1,:] = (idcs[0],vart)

    result = {}

    
    #hist(y-y_predt-y_pred,41)

    #print(vardat)
    result['variance'] = var0
    vsa = 1-(vardat[0,1])/var0
    #print(labels[int(vardat[0,0])],vsa*100,vsa)
    result[labels[int(vardat[0,0])]] = vsa
    for i in range(1,vardat.shape[0]):
        #print(vardat[i-1,1],vardat[i,1])
        if relative:
            var0 = vardat[i-1,1]
            vs = 1-(vardat[i,1])/var0
            result[labels[int(vardat[i,0])]]=vs
        else:
            vs = 1-(vardat[i,1])/var0
            result[labels[int(vardat[i,0])]]=vs-vsa
            vsa = vs

    return(result)

In [54]:
def varanalysis(data, y, labels, method, sel = None):
    ''' Provide analysis of variance for  quantity y
    '''
    if sel is None:
        sel = np.arange(data[labels[0]].shape[0])
    datt = column_stack(tuple(data[l][sel] for l in labels))
    pddat = pd.DataFrame(data=datt, columns = labels)
    varas = method(pddat, y[sel])
    return varas
varanalysis(dataT, mesT[:,3], labelsmod, LinReg_varexplain_B)

{'A1': 0.8907024102083835,
 'Dss': 0.06403969929088948,
 'Rneck': 0.0004761509705282843,
 'variance': 0.20150131740383248}

In [55]:
sel = dataT['nPSD'] == 2.0
varanalysis(dataT, mesT[:,3], labelsmod, LinReg_varexplain_B, sel)

{'A1': 0.7668807171570351,
 'Dss': 0.16563615572185353,
 'Rneck': 0.00415455637670141,
 'variance': 0.16444568073767576}

In [56]:
quants = ['Vsoma', 'Vsp', 'Vsp-Vsh', 'Vsp/Vsh', 'Vsh/Vsp',
         'C','Sv', 'Sc', 'St']
Ys = {}
Ys[quants[0]] = mesT[:,3]
Ys[quants[1]] = mesT[:,1]
Ys[quants[2]] = mesT[:,1]-mesT[:,2]
Ys[quants[3]] = mesT[:,1]/mesT[:,2]
Ys[quants[4]] = mesT[:,2]/mesT[:,1]
Ys[quants[5]] = mesT[:,4]
Ys[quants[6]] = mes1[:,1]/mes01[:,1]
Ys[quants[7]] = mes1[:,4]/mes01[:,4]
Ys[quants[8]] = times/times0

In [57]:
namesfuns = ['LinReg_B', 'LinReg_F', 'LinReg_F_relative']


In [58]:
def multipleanalysis(data, disd, Ys, labels):
    varas = {}
    names = ['LinReg_B', 'LinReg_F', 'LinReg_F_relative']
    LinregFrelative =  lambda x, y: LinReg_varexplain_B(x,y, relative = True)
    for i, method in enumerate([LinReg_varexplain_B,
                                LinReg_varexplain_F,
                                LinregFrelative]):
        vart = {}
        for quant in quants[:6]:
            vart[quant] = varanalysis(data, Ys[quant], labels, method) 
            sel = data['nPSD'] == 2.0
            vart[quant+'DiS'] = varanalysis(data, Ys[quant], labels, 
                                      method, sel) 
                                       
        for quant in quants[6:]:
            vart[quant] = varanalysis(disd, Ys[quant], labels, method) 
        
        varas[names[i]] = vart
    return varas
                               

In [59]:
pddismodel.keys()

Index(['A1', 'maxDhead', 'Dss', 'Lneck', 'Lhead', 'meanDneck_trunc70pc', 'A2'], dtype='object')

In [60]:
resultsmodel = multipleanalysis(dataT,pddis, Ys, labelsmod)
resultsmorph = multipleanalysis(dataT,pddis, Ys, labelmorph)


In [61]:
labfun = namesfuns[0]
Vidcs = resultsmodel[labfun]

In [62]:
labels = labelsmod
for labfun in namesfuns:

    print(10*'*'+labfun+'*'*10)
    Vidcs = resultsmodel[labfun]
    print('Quantity \t'+' \t'.join([label for label in labels+['Total variance']]))
    for key in Vidcs.keys():
        Vidcst = Vidcs[key]
        formattext = '{:.4f} \t'*(len(labels)+1)+' '
        vst = np.array([Vidcst[k] for k in labels+['variance']])
        print('{} \t'.format(key)+formattext.format(*vst) )

**********LinReg_B**********
Quantity 	A1 	Dss 	Rneck 	Total variance
Vsoma 	0.8907 	0.0640 	0.0005 	0.2015 	 
VsomaDiS 	0.7669 	0.1656 	0.0042 	0.1644 	 
Vsp 	0.6021 	0.0002 	0.2016 	39.3462 	 
VspDiS 	0.4683 	0.0032 	0.3942 	59.4472 	 
Vsp-Vsh 	0.3478 	0.0007 	0.2598 	20.8225 	 
Vsp-VshDiS 	0.2544 	0.0007 	0.5127 	38.2093 	 
Vsp/Vsh 	0.0040 	0.0082 	0.9687 	2.3418 	 
Vsp/VshDiS 	0.0042 	0.0128 	0.9682 	2.3172 	 
Vsh/Vsp 	0.0255 	0.0064 	0.7356 	0.0480 	 
Vsh/VspDiS 	0.0009 	0.0223 	0.7181 	0.0579 	 
C 	0.3024 	0.0001 	0.0940 	0.8904 	 
CDiS 	0.4514 	0.0001 	0.0979 	0.5888 	 
Sv 	0.0356 	0.0157 	0.3683 	0.0030 	 
Sc 	0.2739 	0.0128 	0.2236 	0.0083 	 
St 	0.0836 	0.3887 	0.0104 	0.0062 	 
**********LinReg_F**********
Quantity 	A1 	Dss 	Rneck 	Total variance
Vsoma 	0.8907 	0.0640 	0.0005 	0.2015 	 
VsomaDiS 	0.7669 	0.1467 	0.0091 	0.1644 	 
Vsp 	0.6021 	0.0004 	0.1881 	39.3462 	 
VspDiS 	0.4683 	0.0094 	0.3787 	59.4472 	 
Vsp-Vsh 	0.3246 	0.0000 	0.2598 	20.8225 	 
Vsp-VshDiS 	0.2444 	

In [63]:
labels = labelsmod
for labfun in namesfuns:

    print(10*'*'+labfun+'*'*10)
    Vidcs = resultsmodel[labfun]
    print('Quantity '+' '.join([label for label in labels+['Total variance']]))
    for key in Vidcs.keys():
        Vidcst = Vidcs[key]
        formattext = '{:.4f} '*(len(labels)+1)+' '
        vst = np.array([Vidcst[k] for k in labels+['variance']])
        print('{} '.format(key)+formattext.format(*vst) )

**********LinReg_B**********
Quantity A1 Dss Rneck Total variance
Vsoma 0.8907 0.0640 0.0005 0.2015  
VsomaDiS 0.7669 0.1656 0.0042 0.1644  
Vsp 0.6021 0.0002 0.2016 39.3462  
VspDiS 0.4683 0.0032 0.3942 59.4472  
Vsp-Vsh 0.3478 0.0007 0.2598 20.8225  
Vsp-VshDiS 0.2544 0.0007 0.5127 38.2093  
Vsp/Vsh 0.0040 0.0082 0.9687 2.3418  
Vsp/VshDiS 0.0042 0.0128 0.9682 2.3172  
Vsh/Vsp 0.0255 0.0064 0.7356 0.0480  
Vsh/VspDiS 0.0009 0.0223 0.7181 0.0579  
C 0.3024 0.0001 0.0940 0.8904  
CDiS 0.4514 0.0001 0.0979 0.5888  
Sv 0.0356 0.0157 0.3683 0.0030  
Sc 0.2739 0.0128 0.2236 0.0083  
St 0.0836 0.3887 0.0104 0.0062  
**********LinReg_F**********
Quantity A1 Dss Rneck Total variance
Vsoma 0.8907 0.0640 0.0005 0.2015  
VsomaDiS 0.7669 0.1467 0.0091 0.1644  
Vsp 0.6021 0.0004 0.1881 39.3462  
VspDiS 0.4683 0.0094 0.3787 59.4472  
Vsp-Vsh 0.3246 0.0000 0.2598 20.8225  
Vsp-VshDiS 0.2444 0.0017 0.5127 38.2093  
Vsp/Vsh 0.0042 0.0080 0.9687 2.3418  
Vsp/VshDiS 0.0038 0.0127 0.9682 2.3172  
Vsh/Vsp